In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
target_labels = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_mosaic_virus', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
# Load the images and labels
filename = []
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        filename.append(img_file)
        X.append(img_array)
        y.append(target_labels.index(label))

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [7]:
X.shape

(26116, 128, 128, 3)

In [8]:
y.shape

(26116, 14)

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, filename_train, filename_test = train_test_split(X, y, filename, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(20892, 128, 128, 3)

In [11]:
X_test.shape

(5224, 128, 128, 3)

In [12]:
y_train.shape

(20892, 14)

In [13]:
y_test.shape

(5224, 14)

In [14]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [15]:
len(target_labels)

14

In [16]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

2023-06-01 17:14:09.659962: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/10
653/653 [==============================] - 198s 301ms/step - loss: 1.2376 - accuracy: 0.5864 - val_loss: 1.2796 - val_accuracy: 0.5840
Epoch 2/10
653/653 [==============================] - 256s 392ms/step - loss: 0.6242 - accuracy: 0.7863 - val_loss: 0.6764 - val_accuracy: 0.7651
Epoch 3/10
653/653 [==============================] - 265s 406ms/step - loss: 0.4682 - accuracy: 0.8404 - val_loss: 0.5775 - val_accuracy: 0.7992
Epoch 4/10
653/653 [==============================] - 280s 429ms/step - loss: 0.3613 - accuracy: 0.8778 - val_loss: 1.2584 - val_accuracy: 0.6476
Epoch 5/10
653/653 [==============================] - 284s 435ms/step - loss: 0.3163 - accuracy: 0.8953 - val_loss: 0.2959 - val_accuracy: 0.9012
Epoch 6/10
653/653 [==============================] - 287s 439ms/step - loss: 0.2654 - accuracy: 0.9088 - val_loss: 0.9283 - val_accuracy: 0.7155
Epoch 7/10
653/653 [==============================] - 291s 446ms/step - loss: 0.2260 - accuracy: 0.9217 - val_loss: 0.4507 -

In [18]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 84.30%


In [19]:
cnn_scores

[0.4843885898590088, 0.84303218126297]

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

164/164 [==============================] - 11s 67ms/step
Accuracy: 84.30%
Precision: 87.69%
Recall: 84.30%
F1 score: 84.38%


In [21]:
from tabulate import tabulate

# Create a DataFrame with true labels, predicted labels, and target labels
df = pd.DataFrame({'Filename': filename_test,
                   'True Labels': [target_labels[label] for label in y_test_classes],
                   'Predicted Labels': [target_labels[label] for label in y_pred]})

# Convert DataFrame to tabular format
table = tabulate(df, headers='keys', tablefmt='psql')

# Print the table
print(table)

# Save the DataFrame to a CSV file
df.to_csv('labels_predictions-cnn-apel+tomat.csv', index=False)

+------+-----------------------------------------------------------------------------+-----------------------------------------------+-----------------------------------------------+
|      | Filename                                                                    | True Labels                                   | Predicted Labels                              |
|------+-----------------------------------------------------------------------------+-----------------------------------------------+-----------------------------------------------|
|    0 | ba2bdb7a-35c7-4842-a8e0-af3eac603cad___GCREC_Bact.Sp 6034.JPG               | Tomato___Bacterial_spot                       | Tomato___Bacterial_spot                       |
|    1 | eaf64b81-ac2a-4c85-8c15-da1986a12b7a___GCREC_Bact.Sp 3683.JPG               | Tomato___Bacterial_spot                       | Tomato___Bacterial_spot                       |
|    2 | 4eca7d70-60ff-41cc-8884-e4844f23abeb___GHLB2 Leaf 8641_flipLR.JPG           

In [22]:
# Filter rows where true label and predicted label do not match
false_predictions = df[df['True Labels'] != df['Predicted Labels']]

# Convert DataFrame to tabular format
table = tabulate(false_predictions, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table)

# Save the false predictions to a CSV file
false_predictions.to_csv('false_predictions-cnn-apel+tomat.csv', index=False)

False Predictions:
+------+----------------------------------------------------------------------------+-----------------------------------------------+-----------------------------------------------+
|      | Filename                                                                   | True Labels                                   | Predicted Labels                              |
|------+----------------------------------------------------------------------------+-----------------------------------------------+-----------------------------------------------|
|    5 | cd83759b-c7da-4406-8794-9b311ab05871___Matt.S_CG 6684.JPG                  | Tomato___Septoria_leaf_spot                   | Tomato___Target_Spot                          |
|    9 | 0f0fc0ee-b19b-4f90-ab41-8f693131f57d___RS_HL 9890_180deg.JPG               | Tomato___healthy                              | Tomato___Target_Spot                          |
|   16 | 2796069c-4173-4395-ae06-0f2de4f3bc6f___YLCV_NREC 2532.JPG     

In [23]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

653/653 [==============================] - 46s 70ms/step


# rf

In [24]:
import time

In [25]:
# import the Random Forest classifier
from RF_Manual import RandomForest

In [26]:
# Train a Random Forest classifier on the extracted features
start_time = time.time()
rf_classifier = RandomForest(n_trees=100, max_depth=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

# count time for training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 2255.953644990921 seconds


In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

164/164 [==============================] - 7s 45ms/step
Random Forest Classifier Accuracy: 95.06%
Random Forest Classifier Precision: 0.95
Random Forest Classifier Recall: 0.95
Random Forest Classifier F1 Score: 0.95


In [28]:
# Create a DataFrame with true labels and predicted labels
df_rf = pd.DataFrame({'Filename': filename_test,
                      'True Labels': [target_labels[label] for label in y_test_classes],
                      'Predicted Labels': [target_labels[label] for label in rf_predictions]})

# Convert DataFrame to tabular format
table_rf = tabulate(df_rf, headers='keys', tablefmt='psql')

# Print the table
print(table_rf)

# Save the DataFrame to a CSV file
df_rf.to_csv('labels_predictions-rf-apel+tomat.csv', index=False)

+------+-----------------------------------------------------------------------------+-----------------------------------------------+-----------------------------------------------+
|      | Filename                                                                    | True Labels                                   | Predicted Labels                              |
|------+-----------------------------------------------------------------------------+-----------------------------------------------+-----------------------------------------------|
|    0 | ba2bdb7a-35c7-4842-a8e0-af3eac603cad___GCREC_Bact.Sp 6034.JPG               | Tomato___Bacterial_spot                       | Tomato___Bacterial_spot                       |
|    1 | eaf64b81-ac2a-4c85-8c15-da1986a12b7a___GCREC_Bact.Sp 3683.JPG               | Tomato___Bacterial_spot                       | Tomato___Bacterial_spot                       |
|    2 | 4eca7d70-60ff-41cc-8884-e4844f23abeb___GHLB2 Leaf 8641_flipLR.JPG           

In [29]:
# Filter rows where true label and predicted label do not match
false_predictions_rf = df_rf[df_rf['True Labels'] != df_rf['Predicted Labels']]

# Convert DataFrame to tabular format
table_rf = tabulate(false_predictions_rf, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table_rf)

# Save the false predictions to a CSV file
false_predictions_rf.to_csv('false_predictions-rf-apel+tomat.csv', index=False)

False Predictions:
+------+-----------------------------------------------------------------------------+-----------------------------------------------+-----------------------------------------------+
|      | Filename                                                                    | True Labels                                   | Predicted Labels                              |
|------+-----------------------------------------------------------------------------+-----------------------------------------------+-----------------------------------------------|
|   34 | 1bcc3b5d-7e00-43fb-983e-95ae271a715b___FREC_Scab 3061_90deg.JPG             | Apple___Apple_scab                            | Apple___Black_rot                             |
|   52 | 1c980f09-7e37-4e57-96e5-d18783691134___Com.G_TgS_FL 9908_new30degFlipLR.JPG | Tomato___Target_Spot                          | Tomato___Spider_mites Two-spotted_spider_mite |
|   66 | 879bb942-2577-425e-8781-9dfb540d98ac___GCREC_Bact.Sp 6072